In [1]:
from ipyparallel import Client
rc = Client(profile='pucv-cluster')
len(rc)

95

In [2]:
bview = rc.load_balanced_view()
dview = rc[:]

In [3]:
import csv
import pandas as pd
import unicodedata
import unidecode
import re

rule3_regex = re.compile(r"[^\"|\|“|”|\'|„|“|‚|‘|»|«|」|「|『|』|‹|›|„|‚]")

In [4]:
with dview.sync_imports():
    import re

importing re on engine(s)


In [5]:
def strip_accents(text):
    aux_text = text.replace("á","a")
    aux_text = aux_text.replace("é","e")
    aux_text = aux_text.replace("í","i")
    aux_text = aux_text.replace("ó","o")
    aux_text = aux_text.replace("ú","u")
    aux_text = aux_text.replace("ñ","n")
    
    return aux_text

def isOnIt(word, sentence, flag_inductive = False):
    temp_words = word.upper().split(" ")
    if (len(temp_words ) > 1) and not(flag_inductive):
        regex_str = "\\b({}|{})\\b".format(word.upper()," ".join(temp_words[1:]))
    else:
        regex_str = "\\b({})\\b".format(word.upper())
    x = re.findall(regex_str,str(sentence).upper())
    if x:
        return True
    else:
        return False
    
    tokens = strip_accents(str(sentence).lower()).upper().split()
    
    upper_tokens = tokens
    
    """for token in tokens:
        upper_tokens.append(token.upper())"""
    
    if strip_accents(word).upper() in upper_tokens:
        if not str(sentence) in ['nan','--','/','-','?',',','No']:
            if str(strip_accents(word).upper()) in upper_tokens:
                return True
            else: 
                return False
            
def rule1(sentence):
    return ":" in str(sentence)

def rule2(sentence):
    flag = False
    sent =  str(sentence)
    if ': \"' in sent:
        flag = True
    elif ": \'" in sent:
        flag = True
        
    elif ':\"' in sent:
        flag = True
    elif ":\'" in sent:
        flag = True
        
    elif ": “" in sent:
        flag = True
    elif ":“" in sent:
        flag = True
        
    elif ": „" in sent:
        flag = True
    elif ": „" in sent:
        flag = True
        
    elif ": ‚" in sent:
        flag = True
    elif ":‚" in sent:
        flag = True
        
    elif ": ‘" in sent:
        flag = True
    elif ":‘" in sent:
        flag = True
        
    elif ": «" in sent:
        flag = True
    elif ":«" in sent:
        flag = True
    
    elif ": 「" in sent:
        flag = True
    elif ":「" in sent:
        flag = True
    
    elif ": 『" in sent:
        flag = True
    elif ":『" in sent:
        flag = True
    
    elif ": ‹" in sent:
        flag = True
    elif ":‹" in sent:
        flag = True
    
    return flag
    
    

def rule3(sentence):
    aux_text = rule3_regex.sub("",str(sentence))
    return len(aux_text) >= 2

def rule4(sentence):
    aux_text = rule3_regex.sub("",str(sentence))
    return len(aux_text) >= 1

In [6]:
test = "Terremoto sacude las costas de Jamaica y Cuba. El sismo tuvo una magnitud de 7.7. Además, Minsal descartó presencia de Coronavirus en Chile tras el estudio a pacientes con síntomas. Y el Presidente Piñera llegó hasta las zonas más afectadas por las lluvias y aluviones en el norte de nuestro país."
test

'Terremoto sacude las costas de Jamaica y Cuba. El sismo tuvo una magnitud de 7.7. Además, Minsal descartó presencia de Coronavirus en Chile tras el estudio a pacientes con síntomas. Y el Presidente Piñera llegó hasta las zonas más afectadas por las lluvias y aluviones en el norte de nuestro país.'

In [7]:
isOnIt("Sebastián Piñera",test)

True

In [8]:
print("")
rule1("Esto es una prueba: Para ver si detecta los dos puntos Sebastián Piñera".upper())

True

In [9]:
rule2("Esto es una prueba: Para ver si detecta los dos puntos")

False

In [10]:
rule2("Esto es una prueba: 'Para ver si detecta los dos puntos'")

True

In [11]:
rule3("Esto es una prueba: 'Para ver si detecta los dos puntos'")

True

In [12]:
cFile01 = 'COVID-post_enero-julio_FBK-DEF_final.csv'
total_actores = []
total_tipos = []
df_test = pd.read_csv(cFile01, sep=';', low_memory=False,encoding='utf-8-sig')
print(df_test.info())

df_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304002 entries, 0 to 304001
Data columns (total 9 columns):
Date                304002 non-null object
Country             304002 non-null object
MediaOutlet         304002 non-null object
MediaOrigin         304002 non-null object
Message             296906 non-null object
LinkText            293995 non-null object
Verbs R2            88573 non-null object
Actors R1           128441 non-null object
Actors R1 - Type    128455 non-null object
dtypes: object(9)
memory usage: 20.9+ MB
None


,Date,Country,MediaOutlet,MediaOrigin,Message,LinkText,Verbs R2,Actors R1,Actors R1 - Type
0,"Jan 2, 2020 @ 19:19:38.000",United States,Los Angeles Times,Print,The Trump administration has announced a tempo...,"Trump administration imposed ban on some, but ...",announced,Trump,Political and Partisian Source
1,"Jan 2, 2020 @ 22:39:39.000",United States,CBS US,Radio,"The U.S. faces an epidemic of cyberattacks, wi...",Ransomware attack shuts down some Michigan sch...,NaN,NaN,NaN
2,"Jan 3, 2020 @ 01:40:00.000",United States,Los Angeles Times,Print,California has expanded a quarantine zone in S...,California expands local quarantine to halt ci...,state,NaN,NaN
3,"Jan 3, 2020 @ 08:16:13.000",Spain,24hTVE,TV,"La decisión, que entrará en vigor en 30 días, ...",Prohíben en Estados Unidos los cartuchos para ...,NaN,"Donald Trump,Trump","Political and Partisian Source,Political and P..."
4,"Jan 3, 2020 @ 14:40:00.000",Chile,Elmostrador.cl,Online,CULTURA| La escasez hídrica producto del cambi...,Académica chilena en Nature: Más inversión en ...,NaN,NaN,NaN


In [13]:
total_rules1 = []
total_rules2 = []
total_rules3 = []
total_rules4 = []
msgs = df_test['Message'].values
for i in (range(len(msgs))):
    total_rules1.append(rule1(msgs[i]))
    total_rules2.append(rule2(msgs[i]))
    total_rules3.append(rule3(msgs[i]))
    total_rules4.append(rule4(msgs[i]))
    
#total_rules1
df_test["rule 1"] = pd.Series(total_rules1)
df_test["rule 2"] = pd.Series(total_rules2)
df_test["rule 3"] = pd.Series(total_rules3)
df_test["rule 4"] = pd.Series(total_rules4)

In [14]:
df_test.to_csv(cFile01.replace("EM_NV_NEW","EM_NV_RULES"),sep=";",index=False,encoding='utf-8-sig')

In [15]:
cFile01 = 'COVID-post_enero-julio_IGRAM-DEF_final.csv'
total_actores = []
total_tipos = []
df_test = pd.read_csv(cFile01, sep=';', low_memory=False,encoding='utf-8-sig')
print(df_test.info())

df_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33970 entries, 0 to 33969
Data columns (total 9 columns):
Date                33970 non-null object
Country             33970 non-null object
MediaOutlet         33970 non-null object
MediaOrigin         33970 non-null object
Message             33967 non-null object
ImageText           10456 non-null object
Verbs R2            18327 non-null object
Actors R1           22870 non-null object
Actors R1 - Type    22870 non-null object
dtypes: object(9)
memory usage: 2.3+ MB
None


,Date,Country,MediaOutlet,MediaOrigin,Message,ImageText,Verbs R2,Actors R1,Actors R1 - Type
0,"Jan 2, 2020 @ 17:36:00.000",United States,Los Angeles Times,Print,The Trump administration announced a temporary...,NaN,announced,"White House,Trump","Political and Partisian Source,Political and P..."
1,"Jan 8, 2020 @ 12:15:00.000",United States,Los Angeles Times,Print,"An estimated 46,000 Haitians reside in the U.S...",NaN,estimated,NaN,NaN
2,"Jan 12, 2020 @ 08:44:00.000",United Kingdom,Sky News,TV,Knife crime offences are at a record high but ...,NaN,"said,spoke",NaN,NaN
3,"Jan 13, 2020 @ 14:02:37.000",Chile,Mega,TV,La Organización Mundial de la Salud #OMS y las...,Nuevo virus logra salir de China y se registra...,"Según el,anunciaron","autoridades,Wuhan,familia,Organización Mundial...","Political and Partisian Source,Political and P..."
4,"Jan 13, 2020 @ 19:52:08.000",Chile,Canal 13,TV,Una de las expositoras más esperadas del #Cong...,"CONGRESO T13 ""Está documentado que los hombres...",expuso,Escritora,Performer-Artist or Celebrity Source


In [16]:
total_rules1 = []
total_rules2 = []
total_rules3 = []
total_rules4 = []
msgs = df_test['Message'].values
for i in (range(len(msgs))):
    total_rules1.append(rule1(msgs[i]))
    total_rules2.append(rule2(msgs[i]))
    total_rules3.append(rule3(msgs[i]))
    total_rules4.append(rule4(msgs[i]))
    
#total_rules1
df_test["rule 1"] = pd.Series(total_rules1)
df_test["rule 2"] = pd.Series(total_rules2)
df_test["rule 3"] = pd.Series(total_rules3)
df_test["rule 4"] = pd.Series(total_rules4)

In [17]:
df_test.to_csv(cFile01.replace("EM_NV_NEW","EM_NV_RULES"),sep=";",index=False,encoding='utf-8-sig')

In [18]:
cFile01 = 'COVID-post_enero-julio_TWR-DEF_final.csv'
total_actores = []
total_tipos = []
df_test = pd.read_csv(cFile01, sep=';', low_memory=False,encoding='utf-8-sig')
print(df_test.info())

df_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314768 entries, 0 to 314767
Data columns (total 8 columns):
Date                314768 non-null object
Country             314768 non-null object
MediaOutlet         314768 non-null object
MediaOrigin         314768 non-null object
Message             314768 non-null object
Verbs R2            78222 non-null object
Actors R1           119771 non-null object
Actors R1 - Type    119773 non-null object
dtypes: object(8)
memory usage: 19.2+ MB
None


,Date,Country,MediaOutlet,MediaOrigin,Message,Verbs R2,Actors R1,Actors R1 - Type
0,"Aug 1, 2020 @ 15:32:45.000",Spain,Huffpost ES,Online,"Madrid, la gran ciudad europea con mayor exces...",NaN,NaN,NaN
1,"Aug 1, 2020 @ 14:04:00.000",Spain,Huffpost ES,Online,"Rifirrafe entre Fauci, el 'Fernando Simón de E...",NaN,Fernando Simón,Political and Partisian Source
2,"Aug 1, 2020 @ 12:43:39.000",Spain,Huffpost ES,Online,El esperanzador mensaje de una investigadora e...,NaN,NaN,NaN
3,"Aug 1, 2020 @ 11:34:04.000",Germany,BILD,Print,Coronavirus-News im Live-Ticker - Demonstrante...,NaN,NaN,NaN
4,"Aug 1, 2020 @ 10:54:34.000",Germany,BILD,Print,„Keine Touristen in Quarantäne-Hotel“ - Hotelb...,NaN,NaN,NaN


In [19]:
total_rules1 = []
total_rules2 = []
total_rules3 = []
total_rules4 = []
msgs = df_test['Message'].values
for i in (range(len(msgs))):
    total_rules1.append(rule1(msgs[i]))
    total_rules2.append(rule2(msgs[i]))
    total_rules3.append(rule3(msgs[i]))
    total_rules4.append(rule4(msgs[i]))
    
#total_rules1
df_test["rule 1"] = pd.Series(total_rules1)
df_test["rule 2"] = pd.Series(total_rules2)
df_test["rule 3"] = pd.Series(total_rules3)
df_test["rule 4"] = pd.Series(total_rules4)

In [20]:
df_test.to_csv(cFile01.replace("EM_NV_NEW","EM_NV_RULES"),sep=";",index=False,encoding='utf-8-sig')